In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from time import sleep
import requests as r 
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np 
import numpy as array 
import chromedriver_autoinstaller 
import gspread

In [ ]:
solution_dict  ={}
no_of_solutions = 1
tags_frame = {}
no_of_rows = 1
comments_frame = {}
no_of_com = 1
i = 1
page_starter = input("From which page do you want to start?: ")
page_starter = int(page_starter)
page_counter = input("HOW MANY PAEGES YOU WANT TO SCRAPE?(min 2)")
page_counter = int(page_counter)
while i < page_counter:
    # Fetching the main page for solutions 
    url = f"https://patient-innovation.com/browse?vid=All&sort_by=created&page={str(page_starter)}"
    print(url)
    res = r.get(url)
    data = res.text
    soup = bs(data, 'html.parser')
    ul = soup.find('ul',{'class':"latestSolutionsHome search-cards-list clearfix"})
    solutions = ul.find_all('li')
    sol_links = []
    for solution in solutions:
        try:
            link = solution.find('a').get('href')
            if "post/" in link:
                print(link)
                sol_links.append(link)
        except:
            pass
    # Going through solutions 
    for sol_link in sol_links:
        res = r.get(f"https://patient-innovation.com{sol_link}")
        print(f"https://patient-innovation.com{sol_link}")
        data = res.text
        soup = bs(data, 'html.parser')
        stats = soup.find('ul',{'class':'solution-stats-list'})
        title = soup.find('h1',{'class':"solution-title"}).text.strip()
        views = stats.find('li').text.strip()
        likes = stats.find('span',{'class':'likes-counter'}).text.strip()
        boughts = stats.find('span',{'class':'boughts-counter'}).text.strip()
        copies = stats.find('span',{'class':'copies-counter'}).text.strip()
        no_of_comments = stats.find_all('li')[4].text.strip()
        dates = soup.find('p',{'class':"solution-desc-info"}).text.strip()[-16:-5]
        # About the author 
        about_author = soup.find('div',{'class':{"about-author-content"}}).find('p').text.strip()
        # tags 
        tags = ""
        tags_list = soup.find('ul',{"class":"solution-tags-list"}).find_all('li')
        for tag in tags_list:
            tags_frame[no_of_rows] = [title,tag.text.strip()]
            no_of_rows += 1
        # Comments 
        comments_list = soup.find_all('div',{'class':"solution-comments-content"})
        if len(comments_list)>0: 
            for comment in comments_list:
                comments_frame[no_of_com]= [title, comment.find_all('p')[1].text.strip()]
                no_of_com += 1
        else:
            comments_frame[no_of_com] = [title,"No comments Found"]
            no_of_com +=1 
        about_solution = soup.find('div',{'class':"about-solution-content"}).text.strip()
        solution_dict[no_of_solutions] = [title,dates,likes ,no_of_comments,views, boughts,copies,about_author,about_solution]
        print(solution_dict[no_of_solutions])
        no_of_solutions += 1
        sleep(2)
    i += 1
    page_starter += 1
df = pd.DataFrame.from_dict(solution_dict, orient="index", columns=["Title","Date Posted","Hearts - Like Solution", "No of Comments", "Views","Bag - I brought this solution", "Copy- I made a Copy of this", "About the author",  "About the Solution"])
df.to_csv("Solutions.csv")
tags_df = pd.DataFrame.from_dict(tags_frame, orient="index", columns=["Title","tags"])
tags_df.to_csv("tags.csv")

comments_df = pd.DataFrame.from_dict(comments_frame, orient="index", columns=["Title","Comments"])
comments_df.to_csv("Comments.csv")